## Big Data Challenge
### Two goals

1) Perform the ETL process completely in the cloud and upload a dataframe to an RDS instance

2) Use PySpark or SQL to perform a statistical analysis of selected data.


In [43]:
import pandas as pd
import os
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version


In [44]:
# Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
# !tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
# !pip install -q findspark


In [45]:
DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
load_dotenv()


True

In [46]:
# connect to local postgres database
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:5432/big_data')
# load df to database

In [47]:
from pyspark import SparkContext
sc = SparkContext()

RuntimeError: Java gateway process exited before sending its port number

In [41]:



# Set Environment Variables
import os
os.environ["JAVA_HOME"]="/usr/libexec/java_home"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

os.environ["SPARK_HOME"] = f"/usr/local/share/apache-spark/spark-3.2.0-bin-hadoop3.2"
# /usr/local/Cellar/apache-spark/3.2.0/libexec


# Start a SparkSession
import findspark
findspark.init()


In [42]:
# Start Spark session
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("CloudETLProject").getOrCreate()

from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
val spark = SparkSession.builder.getOrCreate()
val segments = spark.read.option("sep", "\t").csv(url)

spark.sparkContext.addFile(url)



SyntaxError: invalid syntax (<ipython-input-42-6eaadcddbd56>, line 9)

In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/opt/anaconda3/bin/python"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.9-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [3]:
import numpy as np
sc = SparkContext.getOrCreate() 
TOTAL = 10000
dots = sc.parallelize([2.0 * np.random.random(2) - 1.0 for i in range(TOTAL)]).cache()
print("Number of random points:", dots.count())
stats = dots.stats()
print('Mean:', stats.mean())
print('stdev:', stats.stdev())

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.storage.StorageUtils$
	at org.apache.spark.storage.BlockManagerMasterEndpoint.<init>(BlockManagerMasterEndpoint.scala:110)
	at org.apache.spark.SparkEnv$.$anonfun$create$9(SparkEnv.scala:348)
	at org.apache.spark.SparkEnv$.registerOrLookupEndpoint$1(SparkEnv.scala:287)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:336)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:191)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:277)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:460)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at

In [1]:
import findspark from pyspark 


SyntaxError: invalid syntax (<ipython-input-1-31ac8df0487e>, line 1)

In [2]:
import findspark
